In [ ]:
from flask import Blueprint, request, jsonify
from app import db
from app.models import User, File
from flask_jwt_extended import create_access_token, jwt_required
from werkzeug.security import generate_password_hash, check_password_hash
import os
from datetime import timedelta

auth_bp = Blueprint('auth', __name__)


@auth_bp.route('/signup', methods=['POST'])
def signup():
    data = request.get_json()
    username = data['username']
    email = data['email']
    password = generate_password_hash(data['password'])

    new_user = User(username=username, email=email, password=password, role='Client')
    db.session.add(new_user)
    db.session.commit()

    return jsonify({"message": "User created successfully!"}), 201

@auth_bp.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    username = data['username']
    password = data['password']

    user = User.query.filter_by(username=username).first()

    if user and check_password_hash(user.password, password):
        access_token = create_access_token(identity=user.id, fresh=True, expires_delta=timedelta(hours=1))
        return jsonify({"access_token": access_token}), 200
    return jsonify({"message": "Invalid credentials"}), 401

@auth_bp.route('/upload', methods=['POST'])
@jwt_required()
def upload_file():
    current_user = User.query.filter_by(id=get_jwt_identity()).first()
    if current_user.role != 'Ops':
        return jsonify({"message": "Permission denied"}), 403

    file = request.files['file']
    if file and file.filename.endswith(('pptx', 'docx', 'xlsx')):
        filename = os.path.join('uploads', file.filename)
        file.save(filename)
        new_file = File(filename=filename)
        db.session.add(new_file)
        db.session.commit()
        return jsonify({"message": "File uploaded successfully!"}), 201
    return jsonify({"message": "Invalid file type"}), 400

@auth_bp.route('/download-file/<file_id>', methods=['GET'])
@jwt_required()
def download_file(file_id):
    file = File.query.get(file_id)
    if file:
        
        download_url = f'/download/{file.id}'
        return jsonify({"download-link": download_url, "message": "success"}), 200
    return jsonify({"message": "File not found"}), 404

